In [1]:
from os import path
import pandas as pd
from serpapi import GoogleSearch
import datetime

In [2]:
search_term = "data analyst"
search_location = "Geneva, Switzerland"
search_radius = 20
api_key = open("../API keys/serpapi.txt", "r").read()


for num in range(45):

    start = num * 10
    params = {
        "api_key": api_key,
        "device": "desktop",
        "engine": "google_jobs",
        "google_domain": "google.com",
        "q": search_term,
        "hl": "en",
        "gl": "ch",
        "lrad": search_radius,
        "location": search_location,
        #"chips": "date_posted:today",
        "start": start,
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # check if the last search page (i.e., no results)
    try:
        if results['error'] == "Google hasn't returned any results for this query.":
            break
    except KeyError:
        print(f"Getting SerpAPI data for page: {start}")
    else:
        continue

    # create dataframe of 10 pulled results
    jobs = results['jobs_results']
    jobs = pd.DataFrame(jobs)
    jobs = pd.concat([pd.DataFrame(jobs), 
                    pd.json_normalize(jobs['detected_extensions'])], 
                    axis=1)
    jobs['date_time'] = datetime.datetime.utcnow()

    # concat dataframe
    if start == 0:
        jobs_all = jobs
    else:
        jobs_all = pd.concat([jobs_all, jobs])

    jobs_all['search_term'] = search_term
    jobs_all['search_location'] = search_location

Getting SerpAPI data for page: 0


C:\Users\Ygrek\AppData\Local\Temp\ipykernel_31708\3025872677.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jobs['date_time'] = datetime.datetime.utcnow()


Getting SerpAPI data for page: 10


C:\Users\Ygrek\AppData\Local\Temp\ipykernel_31708\3025872677.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jobs['date_time'] = datetime.datetime.utcnow()


Getting SerpAPI data for page: 20


...

C:\Users\Ygrek\AppData\Local\Temp\ipykernel_31708\3025872677.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jobs['date_time'] = datetime.datetime.utcnow()


Getting SerpAPI data for page: 60


C:\Users\Ygrek\AppData\Local\Temp\ipykernel_31708\3025872677.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jobs['date_time'] = datetime.datetime.utcnow()


Getting SerpAPI data for page: 70


C:\Users\Ygrek\AppData\Local\Temp\ipykernel_31708\3025872677.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jobs['date_time'] = datetime.datetime.utcnow()


In [3]:
jobs_all.drop(columns='detected_extensions',inplace=True)

In [4]:
jobs_all

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,posted_at,schedule_type,date_time,search_term,search_location,commute_time
0,Data analyst (H/F),ROLEX SA,Geneva,via Www.carrieres-Rolex.com,Introduction\n\nPour notre division Système d’...,[{'items': ['Introduction Pour notre division...,"[{'link': 'http://www.rolex.com/', 'text': 'ro...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[9 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIGFuYWx5c3QgKEgvRikiLC...,9 days ago,Full-time,2024-07-23 08:19:10.001464,data analyst,"Geneva, Switzerland",NaN
1,Data Analyst,Technology Staffing Group,Geneva,via LinkedIn,About TSG\n\nTechnology Staffing Group SA is a...,[{'items': ['About TSG Technology Staffing Gr...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[2 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,2 days ago,Full-time,2024-07-23 08:19:10.001464,data analyst,"Geneva, Switzerland",NaN
2,Data Analyst (h/f),Manpower,Geneva,via JobScout24,Pour une manufacture renommée de la place de G...,[{'items': ['Pour une manufacture renommée de ...,[{'link': 'https://www.google.com/search?sca_e...,NaN,"[21 hours ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgKGgvZikiLC...,21 hours ago,Full-time,2024-07-23 08:19:10.001464,data analyst,"Geneva, Switzerland",NaN
3,Data Analyst,Lynceus Partners,Geneva,via Indeed Suisse,"Job Description\n\nData Analyst\n\nParis, Fran...",[{'items': ['Job Description Data Analyst Pa...,"[{'link': 'http://www.lynceus-partners.com/', ...",NaN,"[7 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,7 days ago,Full-time,2024-07-23 08:19:10.001464,data analyst,"Geneva, Switzerland",NaN
4,Procurement Data Analyst,Richemont,Plan-les-Ouates,via Richemont Jobs,CONTEXT\n\nRichemont is looking for a Procurem...,[{'items': ['CONTEXT Richemont is looking for...,"[{'link': 'http://www.richemont.com/', 'text':...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[17 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJQcm9jdXJlbWVudCBEYXRhIEFuYW...,17 days ago,Full-time,2024-07-23 08:19:10.001464,data analyst,"Geneva, Switzerland",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,VIE - Data Analyst Direction Financière H/F,Indosuez Wealth Management,Geneva,via BeBee,Indosuez Wealth Management is a global wealth ...,[{'items': ['Indosuez Wealth Management is a g...,"[{'link': 'https://ca-indosuez.com/', 'text': ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[20 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJWSUUgLSBEYXRhIEFuYWx5c3QgRG...,20 days ago,Full-time,2024-07-23 08:19:47.580931,data analyst,"Geneva, Switzerland",NaN
8,GVO Client Operations Analyst,Capital Group,Geneva,"via Trabajo.org - Stellenangebote, Arbeit",Primary responsibilities\nProvides high level ...,[{'items': ['Primary responsibilities Provides...,"[{'link': 'https://www.capitalgroup.com/', 'te...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[Full-time],eyJqb2JfdGl0bGUiOiJHVk8gQ2xpZW50IE9wZXJhdGlvbn...,NaN,Full-time,2024-07-23 08:19:47.580931,data analyst,"Geneva, Switzerland",NaN
9,Data analyst,ROLEX SA,Geneva,via BeBee,Fermer\n• Nos métiers\n• Apprentissages et sta...,[{'items': ['Fermer • Nos métiers • Apprentiss...,"[{'link': 'http://www.rolex.com/', 'text': 'ro...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[Full-time],eyJqb2JfdGl0bGUiOiJEYXRhIGFuYWx5c3QiLCJodGlkb2...,NaN,Full-time,2024-07-23 08:19:47.580931,data analyst,"Geneva, Switzerland",NaN
0,Data Analyst développeur de Dashboard,Auctae Switzerland Sàrl,Coppet,via BeBee,/ Travailler chez Auctae / Data Analyst dévelo...,[{'items': ['/ Travailler chez Auctae / Data A...,[{'link': 'https://www.google.com/search?sca_e...,NaN,[Full-time],eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgZMOpdmVsb3...,NaN,Full-time,2024-07-23 08:19:49.027676,data analyst,"Geneva, Switzerland",NaN


In [5]:
jobs_all.describe()

,date_time
count,72
mean,2024-07-23 08:19:29.059540
min,2024-07-23 08:19:10.001464
25%,2024-07-23 08:19:17.067011
50%,2024-07-23 08:19:25.186536
75%,2024-07-23 08:19:43.727969
max,2024-07-23 08:19:49.027676


In [6]:
title_columns = jobs_all.columns

if not path.exists("DB_data-analyst.csv") :
    jobs_all.to_csv("DB_data-analyst.csv", index=False, mode="w")

else :
    df_actual = pd.read_csv("DB_data-analyst.csv")

    df_actual = pd.concat([df_actual, jobs_all], ignore_index=True)

    #Drop duplicates
    df_actual.drop_duplicates(subset="description", inplace=True)

    df_actual.to_csv("DB_data-analyst.csv", index=False, mode="w")